In [161]:
import numpy as np
import pandas as pd

In [171]:
df = pd.read_csv('Precio_Municipios.csv')
df.head()

,Entidad,Municipio,CP,Year,Num_Transacciones,Precio_x_m2
0,Guanajuato,Salamanca,36887,2020,1,9066.79
1,Guanajuato,Salamanca,36885,2020,16,9452.25
2,Guanajuato,Salamanca,36883,2020,1,8543.76
3,Guanajuato,Salamanca,36877,2020,1,12610.97
4,Guanajuato,Salamanca,36875,2020,1,11820.10


In [172]:
df[df['CP'] == 44910]

,Entidad,Municipio,CP,Year,Num_Transacciones,Precio_x_m2


In [186]:
df[df['CP'] == 72703]

,Entidad,Municipio,CP,Year,Num_Transacciones,Precio_x_m2
731,Puebla,Cuautlancingo,72703,2020,15,11329.77
747,Puebla,Cuautlancingo,72703,2019,35,12050.23
761,Puebla,Cuautlancingo,72703,2018,38,10875.15
777,Puebla,Cuautlancingo,72703,2017,24,8693.32
793,Puebla,Cuautlancingo,72703,2016,29,8505.75
810,Puebla,Cuautlancingo,72703,2015,18,8334.02


In [173]:
# Cantidad de cp
len(df['CP'].unique())

161

In [196]:
## Pruebas

prueba = pd.pivot_table(df, values = 'Num_Transacciones', columns = 'Year',index=['Entidad','Municipio','CP']).reset_index()
prueba[prueba['CP'] == 56373 ]

Year,Entidad,Municipio,CP,2015,2016,2017,2018,2019,2020
113,México,Chicoloapan,56373,9.0,42.0,100.0,38.0,118.0,44.0


In [174]:
score_variables = pd.pivot_table(df, values = 'Precio_x_m2', columns = 'Year',index=['Entidad','Municipio','CP']).reset_index()[['Entidad', 'Municipio' ,'CP']]
len(score_variables)

161

In [175]:
score_calificacion = pd.pivot_table(df, values = 'Precio_x_m2', columns = 'Year',index=['Entidad','Municipio','CP']).reset_index()[['Entidad', 'Municipio' ,'CP']]
score_calificacion.head()

Year,Entidad,Municipio,CP
0,Guanajuato,Salamanca,36700
1,Guanajuato,Salamanca,36704
2,Guanajuato,Salamanca,36710
3,Guanajuato,Salamanca,36714
4,Guanajuato,Salamanca,36715


In [176]:
# Precios 15 y 20
# Plusvalia 20/15

precios = df[['Entidad', 'Municipio', 'CP', 'Year', 'Precio_x_m2']]
precios = pd.pivot_table(precios, values = 'Precio_x_m2', columns = 'Year',index=['Entidad','Municipio','CP']).reset_index()
precios = precios[['Entidad','Municipio','CP',2015, 2020]]
precios.rename(columns = {2015: 'precio_15', 2020: 'precio_20'}, inplace = True)
precios['plus_20/15'] = precios['precio_20'] - precios['precio_15']
precios

Year,Entidad,Municipio,CP,precio_15,precio_20,plus_20/15
0,Guanajuato,Salamanca,36700,6954.65,9370.61,2415.96
1,Guanajuato,Salamanca,36704,NaN,NaN,NaN
2,Guanajuato,Salamanca,36710,8375.76,10517.59,2141.83
3,Guanajuato,Salamanca,36714,5571.65,NaN,NaN
4,Guanajuato,Salamanca,36715,9976.22,NaN,NaN
...,...,...,...,...,...,...
156,Puebla,Cuautlancingo,72724,8329.33,12946.02,4616.69
157,Puebla,Cuautlancingo,72725,11195.41,NaN,NaN
158,Puebla,Cuautlancingo,72730,9835.08,11867.63,2032.55
159,Puebla,Cuautlancingo,72733,9234.51,11225.56,1991.05


In [177]:
score_variables = pd.merge(score_variables, precios, how = 'left', on = ['Entidad','Municipio', 'CP'])
len(score_variables)

161

In [178]:
variables = ['precio_15', 'precio_20', 'plus_20/15']
alcaldia = precios.Municipio.unique()

for w in variables:
    nombre = "Cal_"+str(w)
    precios[nombre] = ""


for x in alcaldia:   
    for y in variables:
        
        consulta = precios[precios['Municipio'] == str(x)][[str(y)]].reset_index()
        qs = consulta[y].quantile([.2, .4, .6, .8, 1]).values
        
        nombre = "Cal_"+str(y)
    
        for a,b in consulta.itertuples(index=False):
            if b <= qs[0]:
                precios.loc[a, str(nombre)] = "1"
            elif (b <= qs[1]) & (b > qs[0]):
                precios.loc[a, str(nombre)] = "2"
            elif (b <= qs[2]) & (b > qs[1]):
                precios.loc[a, str(nombre)] = "3"
            elif (b <= qs[3]) & (b > qs[2]):
                precios.loc[a, str(nombre)] = "4"
            elif b <= qs[4]:
                precios.loc[a, str(nombre)] = "5"
            else:
                precios.loc[a, str(nombre)] = "0"
                
precios

Year,Entidad,Municipio,CP,precio_15,precio_20,plus_20/15,Cal_precio_15,Cal_precio_20,Cal_plus_20/15
0,Guanajuato,Salamanca,36700,6954.65,9370.61,2415.96,3,3,4
1,Guanajuato,Salamanca,36704,NaN,NaN,NaN,0,0,0
2,Guanajuato,Salamanca,36710,8375.76,10517.59,2141.83,4,4,3
3,Guanajuato,Salamanca,36714,5571.65,NaN,NaN,1,0,0
4,Guanajuato,Salamanca,36715,9976.22,NaN,NaN,5,0,0
...,...,...,...,...,...,...,...,...,...
156,Puebla,Cuautlancingo,72724,8329.33,12946.02,4616.69,1,4,5
157,Puebla,Cuautlancingo,72725,11195.41,NaN,NaN,5,0,0
158,Puebla,Cuautlancingo,72730,9835.08,11867.63,2032.55,5,3,2
159,Puebla,Cuautlancingo,72733,9234.51,11225.56,1991.05,4,2,1


In [179]:
precios[(precios['CP'] == 72703) | (precios['CP'] == 56373) | (precios['CP'] == 44910) | (precios['CP'] == 55718) | (precios['CP'] == 36723)]

Year,Entidad,Municipio,CP,precio_15,precio_20,plus_20/15,Cal_precio_15,Cal_precio_20,Cal_plus_20/15
9,Guanajuato,Salamanca,36723,9269.37,14319.85,5050.48,5,5,5
113,México,Chicoloapan,56373,8598.18,13194.03,4595.85,4,5,5
133,México,Coacalco de Berriozábal,55718,7122.79,9243.91,2121.12,1,1,3
146,Puebla,Cuautlancingo,72703,8334.02,11329.77,2995.75,2,2,3


In [180]:
fl = precios[['Entidad', 'Municipio' , 'CP' , 'Cal_precio_15', 'Cal_precio_20' ,'Cal_plus_20/15']]

In [181]:
score_calificacion = pd.merge(score_calificacion, fl, how = 'left', on = ['Entidad','Municipio', 'CP'])
len(score_calificacion)

161

In [182]:
# Transacciones
transacciones = df[['Entidad', 'Municipio', 'CP', 'Year', 'Num_Transacciones']]
transacciones = pd.pivot_table(transacciones, values = 'Num_Transacciones', columns = 'Year',index=['Entidad','Municipio','CP']).reset_index()
transacciones  = transacciones[['Entidad', 'Municipio', 'CP', 2015, 2016, 2017, 2018, 2019]]
transacciones.rename(columns = {2015: 'tran_15', 2016: 'tran_16', 2017: 'tran_17', 2018: 'tran_18', 2019: 'tran_19'}, inplace = True)
transacciones

Year,Entidad,Municipio,CP,tran_15,tran_16,tran_17,tran_18,tran_19
0,Guanajuato,Salamanca,36700,13.0,28.0,29.0,18.0,23.0
1,Guanajuato,Salamanca,36704,NaN,NaN,NaN,NaN,1.0
2,Guanajuato,Salamanca,36710,161.0,39.0,122.0,6.0,NaN
3,Guanajuato,Salamanca,36714,2.0,5.0,2.0,1.0,2.0
4,Guanajuato,Salamanca,36715,1.0,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...
156,Puebla,Cuautlancingo,72724,18.0,24.0,16.0,44.0,145.0
157,Puebla,Cuautlancingo,72725,6.0,5.0,3.0,NaN,3.0
158,Puebla,Cuautlancingo,72730,760.0,1016.0,1047.0,170.0,70.0
159,Puebla,Cuautlancingo,72733,33.0,9.0,15.0,7.0,8.0


In [183]:
score_variables = pd.merge(score_variables, transacciones, how = 'left', on = ['Entidad','Municipio', 'CP'])
len(score_variables)

161

In [184]:
variables = ['tran_15', 'tran_16', 'tran_17', 'tran_18', 'tran_19']
alcaldia = transacciones.Municipio.unique()

for w in variables:
    nombre = "Cal_"+str(w)
    transacciones[nombre] = ""


for x in alcaldia:   
    for y in variables:
        
        consulta = transacciones[transacciones['Municipio'] == str(x)][[str(y)]].reset_index()
        qs = consulta[y].quantile([.2, .4, .6, .8, 1]).values
        
        nombre = "Cal_"+str(y)
    
        for a,b in consulta.itertuples(index=False):
            if b <= qs[0]:
                transacciones.loc[a, str(nombre)] = "1"
            elif (b <= qs[1]) & (b > qs[0]):
                transacciones.loc[a, str(nombre)] = "2"
            elif (b <= qs[2]) & (b > qs[1]):
                transacciones.loc[a, str(nombre)] = "3"
            elif (b <= qs[3]) & (b > qs[2]):
                transacciones.loc[a, str(nombre)] = "4"
            elif b <= qs[4]:
                transacciones.loc[a, str(nombre)] = "5"
            else:
                transacciones.loc[a, str(nombre)] = "0"
transacciones

Year,Entidad,Municipio,CP,tran_15,tran_16,tran_17,tran_18,tran_19,Cal_tran_15,Cal_tran_16,Cal_tran_17,Cal_tran_18,Cal_tran_19
0,Guanajuato,Salamanca,36700,13.0,28.0,29.0,18.0,23.0,3,4,4,4,4
1,Guanajuato,Salamanca,36704,NaN,NaN,NaN,NaN,1.0,0,0,0,0,1
2,Guanajuato,Salamanca,36710,161.0,39.0,122.0,6.0,NaN,5,5,5,3,0
3,Guanajuato,Salamanca,36714,2.0,5.0,2.0,1.0,2.0,1,2,1,1,2
4,Guanajuato,Salamanca,36715,1.0,NaN,1.0,NaN,NaN,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,Puebla,Cuautlancingo,72724,18.0,24.0,16.0,44.0,145.0,2,2,2,4,5
157,Puebla,Cuautlancingo,72725,6.0,5.0,3.0,NaN,3.0,2,1,1,0,1
158,Puebla,Cuautlancingo,72730,760.0,1016.0,1047.0,170.0,70.0,5,5,5,5,4
159,Puebla,Cuautlancingo,72733,33.0,9.0,15.0,7.0,8.0,4,2,2,1,2


In [185]:
transacciones[(transacciones['CP'] == 72703) | (transacciones['CP'] == 56373) | (transacciones['CP'] == 44910) | (transacciones['CP'] == 55718) | (transacciones['CP'] == 36723)]

Year,Entidad,Municipio,CP,tran_15,tran_16,tran_17,tran_18,tran_19,Cal_tran_15,Cal_tran_16,Cal_tran_17,Cal_tran_18,Cal_tran_19
9,Guanajuato,Salamanca,36723,39.0,18.0,18.0,11.0,48.0,5,4,4,3,5
113,México,Chicoloapan,56373,9.0,42.0,100.0,38.0,118.0,2,3,4,4,4
133,México,Coacalco de Berriozábal,55718,52.0,90.0,88.0,64.0,80.0,4,4,4,4,4
146,Puebla,Cuautlancingo,72703,18.0,29.0,24.0,38.0,35.0,2,3,3,4,3


In [187]:
fl = transacciones[['Entidad', 'Municipio' , 'CP' , 'Cal_tran_15', 'Cal_tran_16' ,'Cal_tran_17', 'Cal_tran_18' ,'Cal_tran_19']]
score_calificacion = pd.merge(score_calificacion, fl, how = 'left', on = ['Entidad','Municipio', 'CP'])
len(score_calificacion)

161

In [188]:
# Cambio de transacciones
cambio = df[['Entidad', 'Municipio', 'CP', 'Year', 'Num_Transacciones']]
cambio = pd.pivot_table(cambio, values = 'Num_Transacciones', columns = 'Year',index=['Entidad','Municipio','CP']).reset_index()

cambio = cambio[['Entidad', 'Municipio', 'CP', 2016, 2017, 2019]]

cambio['2016-2019'] = cambio[2019] - cambio[2016]
cambio['2017-2019'] = cambio[2019] - cambio[2017]
cambio = cambio[['Entidad', 'Municipio', 'CP', '2016-2019', '2017-2019']]

cambio

Year,Entidad,Municipio,CP,2016-2019,2017-2019
0,Guanajuato,Salamanca,36700,-5.0,-6.0
1,Guanajuato,Salamanca,36704,NaN,NaN
2,Guanajuato,Salamanca,36710,NaN,NaN
3,Guanajuato,Salamanca,36714,-3.0,0.0
4,Guanajuato,Salamanca,36715,NaN,NaN
...,...,...,...,...,...
156,Puebla,Cuautlancingo,72724,121.0,129.0
157,Puebla,Cuautlancingo,72725,-2.0,0.0
158,Puebla,Cuautlancingo,72730,-946.0,-977.0
159,Puebla,Cuautlancingo,72733,-1.0,-7.0


In [189]:
score_variables = pd.merge(score_variables, cambio, how = 'left', on = ['Entidad','Municipio', 'CP'])
len(score_variables)

161

In [190]:
variables = ['2016-2019', '2017-2019']
alcaldia = cambio.Municipio.unique()

for w in variables:
    nombre = "Cal_"+str(w)
    cambio[nombre] = ""


for x in alcaldia:   
    for y in variables:
        
        consulta = cambio[cambio['Municipio'] == str(x)][[str(y)]].reset_index()
        qs = consulta[y].quantile([.2, .4, .6, .8, 1]).values
        
        nombre = "Cal_"+str(y)
    
        for a,b in consulta.itertuples(index=False):
            if b <= qs[0]:
                cambio.loc[a, str(nombre)] = "1"
            elif (b <= qs[1]) & (b > qs[0]):
                cambio.loc[a, str(nombre)] = "2"
            elif (b <= qs[2]) & (b > qs[1]):
                cambio.loc[a, str(nombre)] = "3"
            elif (b <= qs[3]) & (b > qs[2]):
                cambio.loc[a, str(nombre)] = "4"
            elif b <= qs[4]:
                cambio.loc[a, str(nombre)] = "5"
            else:
                cambio.loc[a, str(nombre)] = "0"
cambio

Year,Entidad,Municipio,CP,2016-2019,2017-2019,Cal_2016-2019,Cal_2017-2019
0,Guanajuato,Salamanca,36700,-5.0,-6.0,1,2
1,Guanajuato,Salamanca,36704,NaN,NaN,0,0
2,Guanajuato,Salamanca,36710,NaN,NaN,0,0
3,Guanajuato,Salamanca,36714,-3.0,0.0,2,3
4,Guanajuato,Salamanca,36715,NaN,NaN,0,0
...,...,...,...,...,...,...,...
156,Puebla,Cuautlancingo,72724,121.0,129.0,5,5
157,Puebla,Cuautlancingo,72725,-2.0,0.0,3,4
158,Puebla,Cuautlancingo,72730,-946.0,-977.0,1,1
159,Puebla,Cuautlancingo,72733,-1.0,-7.0,3,3


In [191]:
cambio[(cambio['CP'] == 72703) | (cambio['CP'] == 56373) | (cambio['CP'] == 44910) | (cambio['CP'] == 55718) | (cambio['CP'] == 36723)]

Year,Entidad,Municipio,CP,2016-2019,2017-2019,Cal_2016-2019,Cal_2017-2019
9,Guanajuato,Salamanca,36723,30.0,30.0,5,5
113,México,Chicoloapan,56373,76.0,18.0,5,4
133,México,Coacalco de Berriozábal,55718,-10.0,-8.0,2,2
146,Puebla,Cuautlancingo,72703,6.0,11.0,4,5


In [140]:
fl = cambio[['Entidad', 'Municipio' , 'CP' , 'Cal_2016-2019', 'Cal_2017-2019' ]]
score_calificacion = pd.merge(score_calificacion, fl, how = 'left', on = ['Entidad','Municipio', 'CP'])
len(score_calificacion)

161

In [141]:
score_calificacion

Year,Entidad,Municipio,CP,Cal_precio_15,Cal_precio_20,Cal_plus_20/15,Cal_tran_15,Cal_tran_16,Cal_tran_17,Cal_tran_18,Cal_tran_19,Cal_2016-2019,Cal_2017-2019
0,Guanajuato,Salamanca,36700,8,8,9,8,9,9,9,9,6,7
1,Guanajuato,Salamanca,36704,5,5,5,5,5,5,5,6,5,5
2,Guanajuato,Salamanca,36710,9,9,7,10,10,10,8,5,5,5
3,Guanajuato,Salamanca,36714,6,5,5,6,7,6,6,7,7,8
4,Guanajuato,Salamanca,36715,10,5,5,6,5,6,5,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,Puebla,Cuautlancingo,72724,6,9,10,7,7,7,9,10,10,10
157,Puebla,Cuautlancingo,72725,10,5,5,7,6,6,5,6,8,9
158,Puebla,Cuautlancingo,72730,10,8,6,10,10,10,10,9,6,6
159,Puebla,Cuautlancingo,72733,9,7,7,9,7,7,6,7,8,8


In [142]:
score_variables

Year,Entidad,Municipio,CP,precio_15,precio_20,plus_20/15,tran_15,tran_16,tran_17,tran_18,tran_19,2016-2019,2017-2019
0,Guanajuato,Salamanca,36700,6954.65,9370.61,1.347388,13.0,28.0,29.0,18.0,23.0,-5.0,-6.0
1,Guanajuato,Salamanca,36704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,Guanajuato,Salamanca,36710,8375.76,10517.59,1.255718,161.0,39.0,122.0,6.0,NaN,NaN,NaN
3,Guanajuato,Salamanca,36714,5571.65,NaN,NaN,2.0,5.0,2.0,1.0,2.0,-3.0,0.0
4,Guanajuato,Salamanca,36715,9976.22,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,Puebla,Cuautlancingo,72724,8329.33,12946.02,1.554269,18.0,24.0,16.0,44.0,145.0,121.0,129.0
157,Puebla,Cuautlancingo,72725,11195.41,NaN,NaN,6.0,5.0,3.0,NaN,3.0,-2.0,0.0
158,Puebla,Cuautlancingo,72730,9835.08,11867.63,1.206663,760.0,1016.0,1047.0,170.0,70.0,-946.0,-977.0
159,Puebla,Cuautlancingo,72733,9234.51,11225.56,1.215610,33.0,9.0,15.0,7.0,8.0,-1.0,-7.0
